# 1. 가중치 그래프의 표현

## 1. 인접 행렬

In [21]:
vertex = ['A','B','C','D','E','F','G']
weight =[[None,29,None,None,None,10,None],
         [29,None,16,None,None,None,15],
         [None,16,None,12,None,None,None],
         [None,None,12,None,22,None,18],
         [None,None,None,22,None,27,25],
         [10,None,None,None,27,None,None],
         [None,15,None,18,25,None,None]]
graph = (vertex, weight)

In [23]:
# 인접 행렬에서의 가중치의 합 계산
def weightSum(vlist,W): # 매개변수: 정점 리스트, 인접 행렬
    sum = 0 # 가중치의 합
    for i in range(len(vlist)): # 모든 정점에 대해(i: 0, ... N-1)
        for j in range(i+1,len(vlist)): # 하나의 행에 대해(삼각영역)
            if W[i][j] != None: # 만약 간선이 있으면
                sum +=W[i][j]# sum에 추가
    return sum # 전체 가중치 합을 반환


In [24]:
print('AM : weight sum = ', weightSum(vertex, weight))

AM : weight sum =  174


In [25]:
# 인접 행렬에서의 모든 간선 출력
# 간선은 양쪽 정점 이름과 가중치를 함께 출력
def printAllEdges(vlist, W):
    for i in range(len(vlist)):
        for j in range(i+1, len(W[i])): # 모든 간선 w[i][j]에 대해
            if W[i][j] != None and W[i][j] != 0: # 간선이 있으면
                print("(%s,%s,%d)"%(vlist[i],vlist[j],W[i][j]), end=' ')
    print()

In [26]:
printAllEdges(vertex, weight)

(A,B,29) (A,F,10) (B,C,16) (B,G,15) (C,D,12) (D,E,22) (D,G,18) (E,F,27) (E,G,25) 


## 2. 인접 리스트

In [27]:
# 딕셔너리, 집합, 튜플, 리스트를 이용한 그래프 표현
graph = {'A' : set([('B',29),('F',10)]),
         'B': set([('A',29),('c',16),('G',15)]),
         'C': set([('B',16),('D',12)]),
         'D': set([('C',12),('E',22),('G',18)]),
         'E': set([('D',22),('F',27),('G',25)]),
         'F': set([('A',10),('E',27)]),
         'G': set([('B',15),('D',18),('E',25)])}

In [28]:
def weightSum(graph): # 가중치의 총 합을 구하는 함수
    sum = 0
    for v in graph: # 그래프의 모든 정점 v에 대해: 'A','B',...
        for e in graph[v]: # v의 모든 간선 e에 대해: ('B',29), ...
            sum += e[1] # sum에 추가
    return sum//2 # 하나의 간선이 두 번 더해지므로 2로 나눔

In [29]:
# 인접 리스트에서의 모든 간선 출력
def printAllEdges(graph):
    for v in graph:
        for e in graph[v]:
            print("(%s,%s,%d)"%(v,e[0],e[1]), end=' ')

In [30]:
print ('AL : weight sum =', weightSum(graph))
printAllEdges(graph)

AL : weight sum = 174
(A,B,29) (A,F,10) (B,c,16) (B,A,29) (B,G,15) (C,D,12) (C,B,16) (D,C,12) (D,G,18) (D,E,22) (E,D,22) (E,G,25) (E,F,27) (F,A,10) (F,E,27) (G,E,25) (G,D,18) (G,B,15) 

# 2. 최소비용 신장 트리

* 그래프의 모든 정점들은 연결되어야 한다.
* 연결에 필요한 간선의 가중치 합이 최소가 되어야 한다.
* 사이클은 두 정점을 연결하는 두 가지 결오를 제공하므로 비용 측면에서 바람직하지 않다.  따라서 사이클 없이 (n-1)개의 간선만을 사용해야한다.

최소비용 신장트리(MST: minimum spanning tree)

## 1. Kruskal의 MST 알고리즘

탐욕적인 방법(greedy method)라는 알고리즘 설계에서 중요한 기법임  
" 그 순간에 최적"이라고 생각되는 것을 선택하는 방법  
1. 그래프의 모든 간선을 가중치에 따라 오름차순으로 정렬한다.
2. 가장 가중치가 작은 간선 e를 뽑는다.
3. e를 신장 트리에 넣었을 때 사이클이 생기면 넣지 않고 2번으로 이동한다.
4. 사이클이 생기지 않으면 최소 신장 트리에 삽입한다.
5. n-1개의 간선이 삽입될 때 까지 2번으로 이동한다.

In [53]:
parent = [] # 각 노드의 부모노드 인덱스
set_size = 0 # 전체 집합의 개수

In [54]:
def init_set(nSets) : # 집합의 초기화 함수
    global set_size, parent # 전역변수 사용을 위함
    set_size = nSets; # 집합의 개수
    for i in range(nSets): # 모든 집합에 대해
        parent.append(-1) # 각각이 고유의 집합(부모가 -1)

In [55]:
def find(id): # 정점 id가 속한 집합의 대표번호 탐색
    while (parent[id] >= 0): # 부모가 있는 동안(-1이 아닌 동안)
        id = parent[id] # id를 부모 id로 갱신
    return id # 최종 id 반환. 트리의 맨 위 노드의 id임

In [56]:
def union(s1, s2): # 두 집합을 병함
    global set_size 
    parent[s1] = s2 # s1을 s2에 병합시킴
    set_size=set_size-1

## Kruskal 알고리즘의 구현
사이클 검사 과정 중심
* 초기에는 모든 정점이 각각 고유한 집합이다.
* 최소 가중치 간선 (u,v)가 선택되면 u와 v가 각각 속한 집합을 찾는다. 이때 find(u)와 find(v) 연산을 수행한다.
* 두 집합이 같으면 사이클이 발생하는 상황이므로 이 간선을 버린다.
* 두 집합이 다르면 간선을 삽입하고 집합을 하나로 합친다. 이때 union(u,v) 연산을 사용한다.

kruskal 알고리즘
* 그래프는 인접 행렬을 이용해 나타낸다.
* 간선 리스트에는 간선을 (v1, v2, weight)의 튜플 형태로 저장한다.
* 간선의 정렬이 필요하다. 이를 위해 정렬 알고리즘, 힙 구조(만약 힙을 사용한다면 가중치가 작은 간선을 뽑아야하기 때문에 최소 힙을 사용).  모든 간선들을 파이썬의 리스트에 넣고, 리스트에서 제공하는 정렬함수 sort()를 이용하자. 리스트는 가중치의 내림차순으로 정렬하고 리스트의 맨 뒤에서 하나씩 간선을 꺼내도록 하면 된다.

In [57]:
# kruskal의 최소 비용 신장 트리 프로그램
def MSTKruskal(vertex, adj): # 매개변수: 정점리스트, 인접행렬
    vsize = len(vertex) # 정점의 개수
    init_set(vsize) # 정점 집합 초기화
    eList = [] # 간선 리스트
    
    for i in range(vsize-1): # 모든 간선을 리스트에 넣음
        for j in range(i+1, vsize):
            if adj[i][j] != None:
                eList.append( (i,j,adj[i][j]) ) # 튜플로 저장 시작정점과, 인접정점, 가중치 간선
                
    # 간선 리스트를 가중치의 내림차순으로 정렬: 람다 함수 사용
    eList.sort(key = lambda e: e[2], reverse = True) # reverse를 통해 내림차순으로 해결
    
    edgeAccepted = 0
    while (edgeAccepted < vsize-1): # 정점 수 -1개의 간선
        e = eList.pop(-1) # 가장 작은 가중치를 가진 간선
        uset = find(e[0]) # 두 정점이 속한 집합 번호
        vset = find(e[1])
        
        if uset != vset : # 두 정점이 다른 집합의 원소이면 , eList에서 연결된 정점이 아니기 때문에
            print("간선 추가: (%s, %s, %d)" %
                  (vertex[e[0]], vertex[e[1]], e[2]))
            union(uset, vset) # 두 집합을 합함
            edgeAccepted += 1 # 간선이 하나 추가됨

In [58]:
print("MST By Kruskal's Algorithm")
MSTKruskal(vertex, weight)

MST By Kruskal's Algorithm
간선 추가: (A, F, 10)
간선 추가: (C, D, 12)
간선 추가: (B, G, 15)
간선 추가: (B, C, 16)
간선 추가: (D, E, 22)
간선 추가: (E, F, 27)


In [62]:
# vsize = 7
# for i in range(vsize-1): # 모든 간선을 리스트에 넣음
#         for j in range(i+1, vsize):
#             print(i,j)

0 1
0 2
0 3
0 4
0 5
0 6
1 2
1 3
1 4
1 5
1 6
2 3
2 4
2 5
2 6
3 4
3 5
3 6
4 5
4 6
5 6


## 2. Prim의 MST 알고리즘

Prim()  
MST 트리기준으로 가장 인접한 정접을 찾는 알고리즘
1. 그래프에서 시작 정점을 선택하여 초기 트리를 만든다.
2. 현재 트리의 정점들과 인접한 정점들 중에서 간선의 가중치가 가장 작은 정점 v를 선택한다.
3. 이 정점 v와 이때의 간선을 트리에 추가한다.
4. 모든 정점이 삽입될 때 까지 2번으로 이동한다.

Prim 알고리즘 구현  
* dist[i]는 현재까지 구성된 MST(트리에 속한 정점들)에서 i번째 정점까지의 가장 가까운 거리를 저장한다.  
처음에는 시작 정점의 dist 값은 0이고 나머지 정점의 dist는 무한대가 된다.
* 정점에서 MST에 추가되면서 dist 값들이 갱신된다. 예를 들어, MST에 u가 추가된다면 당연히 dist[u] = 0으로 변경된다.
* u와 인접한 정점 v의 거리 dist[v]도 변경되어야 한다. 즉, 추가되는 정점 u와의 간선(u,v)의 가중치가 기존의 dist[v]보다 작으면  
dist[v]는 이 가중치 값으로 갱신되어야 한다.
* 이 과정은 모든 정점이 MST에 포함될 때 까지 진행한다.

In [65]:
INF = 9999 # 가장 큰 가중치(무한대)
# 현재 트리에 인접한 정점들 중에서 가장 가까운 정점을 찾는 함수
def getMinVertex(dist, selected):
    minv = 0
    mindist = INF
    for v in range(len(dist)):                  # 모든 정점들에 대해
        if not selecte[v] and dist[v]<mindist:  # 선택 안 되었고
            mindist = dist[v]                   # 가중치가 작으면
            minv = v                            # minv 갱신
    return minv                                 # 최소 가중치의 정점 반환

In [82]:
# Prim의 최소 비용 신장 트리 프로그램
def MSTPrim(vertex, adj):
    vsize = len(vertex)
    dist = [INF] * vsize # dist: [INF, INF,...]
    selected = [False] * vsize
    dist[0] = 0
    
    for i in ragne(vsize): # 정점의 수 만큼 반복
        u = getMinVertex(dist, selected)
        selected[u] = True
        print(vertex[u], end=' ')
        
        for v in range(vsize): # 내부루프
            if (adj[u][v] != None): # (u,v) 간선이 있으면 dist[v] 갱신
                if selected[v] == False and adj[u][v]<dist[v]:
                    dist[v] = adj[u][v]
                
    print()

# 3.최단 경로

## 1. Dijkstra의 최단 경로 알고리즘  

1. 특정한 하나의 정점에서 다른 모든 정점으로 가는 최단 경로  
2. 현재까지 알고 있던 최단 경로를 계속해서 갱신  

* 시작 정점v: 최단경로탐색의 시작 정점
* 집합s: 시작 정점 v로부터의 최단경로가 이미 발견된 정점들의 집합
* dist배열: S에 있는 정점만을 거쳐서 다른 정점으로 가는 최단거리를 기록하는 배열.  
Prim의 MST 알고리즘에서와 유사함. dist[v] = 0(시작 정점).

### Dijkstra 알고리즘의 구현  
알고리즘을 구현하기 위해서는 다음의 세 가지 배열(리스트)이 필요하다.  
* dist[]: 시작정점으로부터의 최단경로 거리를 저장
* found[]: 방문한 정점 표시를 위해 사용. 최초 모든 항목이 False
* path[]: 바로 이전 정점을 저장. 이전 정점을 따라 시작 정점까지 가는 경로가 최단 경로임

In [67]:
# dist가 가장 작은 정점 찾기
INF = 9999
def choose_vertex(dist, found):          # 최소 dist 정점을 찾는 함수
    min = INF
    minpos = -1
    for i in range(len(dist)):               # 모든 정점중에서
        if dist[i]<min and found[i]==False:  # 방문하지 않은 최소 dist 정점
            min = dist[i]
            minpos = i
    return minpos # 최소 dist 정점의 인덱스 반환

In [68]:
# 정점 리스트와 인접행렬, 그리고 시작 정점의 인덱스를 매개변수로 받도록 함
def shortest_path_dijkstra(vtx, adj, start):
    vsize = len(vtx)                 # 정점 수
    dist = list(adj[start])          # dist 배열 생성 및 초기화
    path = [start] * vsize           # path 배열 생성 및 초기화
    found = [False] * vsize          # found 배열 생성 및 초기화
    found[start] = True              # 시작정점: 이미 찾아짐
    dist[start] = 0                  # 시작정점의 거리 0
    
    for i in range(vsize):
        print("Step%2d: "%(i+1), dist)    # 단계별 dist[] 출력용
        u = choose_vertex(dist, found)    # 최소 dist 정점 u 탐색
        found[u] = True                   # u는 이제 찾아짐
        
        for w in range(vsize):            # 모든 정점에 대해
            if not found[w]:              # 아직 찾아지지 않았으면
                if dist[u] + adj[u][w] < dist[w]:  # 갱신 조건 검사
                    dist[w] = dist[u] + adj[u][w]  # dist 개신
                    path[w] = u                    # 이전 정점 갱신
                    
    return path                

In [69]:
vertex = ['A','B','C','D','E','F','G']
weight=[[0,7,INF,INF,3,10,INF],
        [7,0,4,10,2,6,INF],
        [INF,4,0,2,INF,INF,INF],
        [INF,10,2,0,11,9,4],
        [3,2,INF,11,0,INF,5],
        [10,6,INF,9,INF,0,INF],
        [INF,INF,INF,4,5,INF,0]]

In [74]:
print("Shortest Path By Dijkstra Algorithm")
start=0 # 시작 정점은 0 번, 'A'로 선택
path = shortest_path_dijkstra(vertex, weight, start)

# 최종 경로를 출력하기 위한 코드
for end in range(len(vertex)):
    if end != start:
        print("[최단경로: %s->%s] %s" %
              (vertex[start], vertex[end], vertex[end]), end='')
        while (path[end] != start):
            print(" <-%s" % vertex[path[end]], end='')
            end = path[end]
        print(" <-%s" % vertex[path[end]])
        
path

Shortest Path By Dijkstra Algorithm
Step 1:  [0, 7, 9999, 9999, 3, 10, 9999]
Step 2:  [0, 5, 9999, 14, 3, 10, 8]
Step 3:  [0, 5, 9, 14, 3, 10, 8]
Step 4:  [0, 5, 9, 12, 3, 10, 8]
Step 5:  [0, 5, 9, 11, 3, 10, 8]
Step 6:  [0, 5, 9, 11, 3, 10, 8]
Step 7:  [0, 5, 9, 11, 3, 10, 8]
[최단경로: A->B] B <-E <-A
[최단경로: A->C] C <-B <-E <-A
[최단경로: A->D] D <-C <-B <-E <-A
[최단경로: A->E] E <-A
[최단경로: A->F] F <-A
[최단경로: A->G] G <-E <-A


[0, 4, 1, 2, 0, 0, 4]

## 2. FLoyd의 최단 경로 알고리즘  
그래프의 모든 정점들 사이의 최단 경로를 구하는 알고리즘

In [80]:
INF = 9999
def printA(A):
    vsize = len(A)     # 현재의 A 행렬을 화면에 출력하는 함수
    print("=================================")
    for i in range(vsize):
        for j in range(vsize):
            if (A[i][j] ==INF): print(" INF ",end='')
            else: print("%4d "%A[i][j], end='')
        print("");
            
def shortest_path_floyd(vertex, adj): # Floyd의 최단경로탐색 함수
    vsize = len(vertex)               # 정점의 개수
    A = list(adj)                     # 주의: 2차원 배열(리스트의 리스트)의 복사
    for i in range(vsize):
        A[i] = list(adj[i])
        
    for k in range(vsize):            # 정점 K를 추가할 때
        for i in range(vsize):
            for j in range(vsize):    # 모든 A[i][j] 갱신
                if (A[i][k] + A[k][j] < A[i][j]):
                    A[i][j] = A[i][k] + A[k][j]
        
        printA(A)                     # 현재 A행렬 출력

In [83]:
vsize = 5
for k in range(vsize):            # 정점 K를 추가할 때
        for i in range(vsize):
            for j in range(vsize):
                print(k,i,j)

0 0 0
0 0 1
0 0 2
0 0 3
0 0 4
0 1 0
0 1 1
0 1 2
0 1 3
0 1 4
0 2 0
0 2 1
0 2 2
0 2 3
0 2 4
0 3 0
0 3 1
0 3 2
0 3 3
0 3 4
0 4 0
0 4 1
0 4 2
0 4 3
0 4 4
1 0 0
1 0 1
1 0 2
1 0 3
1 0 4
1 1 0
1 1 1
1 1 2
1 1 3
1 1 4
1 2 0
1 2 1
1 2 2
1 2 3
1 2 4
1 3 0
1 3 1
1 3 2
1 3 3
1 3 4
1 4 0
1 4 1
1 4 2
1 4 3
1 4 4
2 0 0
2 0 1
2 0 2
2 0 3
2 0 4
2 1 0
2 1 1
2 1 2
2 1 3
2 1 4
2 2 0
2 2 1
2 2 2
2 2 3
2 2 4
2 3 0
2 3 1
2 3 2
2 3 3
2 3 4
2 4 0
2 4 1
2 4 2
2 4 3
2 4 4
3 0 0
3 0 1
3 0 2
3 0 3
3 0 4
3 1 0
3 1 1
3 1 2
3 1 3
3 1 4
3 2 0
3 2 1
3 2 2
3 2 3
3 2 4
3 3 0
3 3 1
3 3 2
3 3 3
3 3 4
3 4 0
3 4 1
3 4 2
3 4 3
3 4 4
4 0 0
4 0 1
4 0 2
4 0 3
4 0 4
4 1 0
4 1 1
4 1 2
4 1 3
4 1 4
4 2 0
4 2 1
4 2 2
4 2 3
4 2 4
4 3 0
4 3 1
4 3 2
4 3 3
4 3 4
4 4 0
4 4 1
4 4 2
4 4 3
4 4 4


In [84]:
vertex = ['A','B','C','D','E','F','G']
weight=[[0,7,INF,INF,3,10,INF],
        [7,0,4,10,2,6,INF],
        [INF,4,0,2,INF,INF,INF],
        [INF,10,2,0,11,9,4],
        [3,2,INF,11,0,INF,5],
        [10,6,INF,9,INF,0,INF],
        [INF,INF,INF,4,5,INF,0]]

In [81]:
path = shortest_path_floyd(vertex, weight)

   0    7  INF  INF    3   10  INF 
   7    0    4   10    2    6  INF 
 INF    4    0    2  INF  INF  INF 
 INF   10    2    0   11    9    4 
   3    2  INF   11    0   13    5 
  10    6  INF    9   13    0  INF 
 INF  INF  INF    4    5  INF    0 
   0    7   11   17    3   10  INF 
   7    0    4   10    2    6  INF 
  11    4    0    2    6   10  INF 
  17   10    2    0   11    9    4 
   3    2    6   11    0    8    5 
  10    6   10    9    8    0  INF 
 INF  INF  INF    4    5  INF    0 
   0    7   11   13    3   10  INF 
   7    0    4    6    2    6  INF 
  11    4    0    2    6   10  INF 
  13    6    2    0    8    9    4 
   3    2    6    8    0    8    5 
  10    6   10    9    8    0  INF 
 INF  INF  INF    4    5  INF    0 
   0    7   11   13    3   10   17 
   7    0    4    6    2    6   10 
  11    4    0    2    6   10    6 
  13    6    2    0    8    9    4 
   3    2    6    8    0    8    5 
  10    6   10    9    8    0   13 
  17   10    6    4    5   1